In [2]:
import numpy as np 
import pandas as pd 
from scipy import signal
import matplotlib.pyplot as plt 
import math
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [3]:
def data_visulization():
    # read dataset file
    for i in range(19):
        df = pd.read_csv('dataset/dataset_' + str(i + 1) + '.txt', sep=',', header=None)
    df_storing = df[df[24] == 1].values
    # In this example code, only accelerometer 1 data (column 1 to 3) is used
    plt.plot(df_storing[:, 0:3])
    plt.show()